# Node Classification

In [1]:
import networkx as nx
import pandas as pd
import numpy as np

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, matthews_corrcoef, confusion_matrix, classification_report
from itertools import product
from sklearn.metrics.pairwise import cosine_similarity
from node2vec import Node2Vec as n2v

In [ ]:
# constants

## Fetch Data

## Create Network

## Apply Node2Vec

## Train Model

## Evaluate Model

## Predictions

## Concluding Remarks


## Resources

---